In [1]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
response = requests.get("https://genius.com/artists/albums?for_artist_page=45&id=Eminem")
response

<Response [403]>

In [23]:
html_string = response.text
document = BeautifulSoup(html_string, 'html.parser')
album_list = document.find_all("a", class_="album_link")
albums = ["The Slim Shady LP", "The Marshall Mathers LP", "The Eminem Show",
         "Encore", "Eminem Presents the Re-Up", "Relapse", "Recovery", "The Marshall Mathers LP 2",
         "Revival", "Kamikaze", "Music to Be Murdered By", "Music to Be Murdered By: Side B", 
          "The Death of Slim Shady (Coup De Grâce)"]
album_links = ["https://genius.com/albums/Eminem/The-slim-shady-lp", "https://genius.com/albums/Eminem/The-marshall-mathers-lp",
              "https://genius.com/albums/Eminem/The-eminem-show", "https://genius.com/albums/Eminem/Encore", 
               "https://genius.com/albums/Eminem/Eminem-presents-the-re-up", "https://genius.com/albums/Eminem/Relapse",
              "https://genius.com/albums/Eminem/Recovery", "https://genius.com/albums/Eminem/The-marshall-mathers-lp-2",
              "https://genius.com/albums/Eminem/Revival", "https://genius.com/albums/Eminem/Kamikaze",
              "https://genius.com/albums/Eminem/Music-to-be-murdered-by", 
               "https://genius.com/albums/Eminem/Music-to-be-murdered-by-side-b",
              "https://genius.com/albums/Eminem/The-death-of-slim-shady-coup-de-grace"]
album_links = []
for a in album_list:
  if a.text in albums:
    album_links.append((a.text, a['href'])) # album links is (album name, link)
    
print(album_list)


[]


In [20]:
eminem_lyrics = {}

for album in album_links:
  eminem_lyrics[album[0]] = {}     # each album has a dictionary
  album_page = requests.get("https://genius.com" + album[1])
  album_html = album_page.text
  doc = BeautifulSoup(album_html, 'html.parser')
  songs_list = doc.find_all("div", class_="chart_row-content")
  
  for s in songs_list:
    a = s.find('a')
    s_title = a.find('h3').get_text(strip=True)
    eminem_lyrics[album[0]][s_title] = {}
    
    s_link = a['href']
    # now use the link s_link to go to the lyrics page
    lyrics_page = requests.get(s_link)
    lyrics_html = lyrics_page.text
    d = BeautifulSoup(lyrics_html, 'html.parser')
    ly = d.find('div', class_="Lyrics__Container-sc-1ynbvzw-1 kUgSbL")
    for br in ly.find_all('br'):
      br.replace_with(' ')
    eminem_lyrics[album[0]][s_title]['lyrics'] = ly.text

ConnectionError: HTTPSConnectionPool(host='genius.comt', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe47f381ad0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [44]:
# to do: need to clean up song titles, need to clean up lyrics (get rid of [verse 1], etc.) 
# basically just clean it all up and then once its all saved, move on to sentiment analysis for each individual song, before
# then moving on to average sentiment for each album. Then, I think I will just stick to the theme of family and compare it
# to trend and values of sentiment
# final project will be something along the lines of how sentiment and family evolve as themes in eminem's discography

print(eminem_lyrics['Encore'])

KeyError: 'Encore'

In [ ]:
lyrics_page = requests.get('https://genius.com/Eminem-breaking-news-skit-lyrics')
lyrics_html = lyrics_page.text
d = BeautifulSoup(lyrics_html, 'html.parser')
ly = d.find('div', class_="Lyrics__Container-sc-1ynbvzw-1 kUgSbL")
for br in ly.find_all('br'):
    br.replace_with('\n')
print(ly.text)